This jupyter notebook is used for rapid development.

In [3]:
# parse the swagger doc
api_methods = []
with open(OPEN_API_FILE) as stream:
    try:
        swagger = yaml.safe_load(stream)
        for path in swagger["paths"]:
            for method in swagger["paths"][path]:
                endpoint = {}
                endpoint["method"] = method.upper()
                endpoint["url"] = path
                api_methods.append(endpoint)
    except yaml.YAMLError as exc:
        print(exc)
        1 / 0

print(json.dumps(api_methods, indent=2))



[
  {
    "method": "GET",
    "url": "/service/ready"
  },
  {
    "method": "GET",
    "url": "/service/liveness"
  },
  {
    "method": "GET",
    "url": "/service/values"
  },
  {
    "method": "GET",
    "url": "/service/values/arch"
  },
  {
    "method": "GET",
    "url": "/service/values/class"
  },
  {
    "method": "GET",
    "url": "/service/values/flag"
  },
  {
    "method": "GET",
    "url": "/service/values/nettype"
  },
  {
    "method": "GET",
    "url": "/service/values/role"
  },
  {
    "method": "GET",
    "url": "/service/values/subrole"
  },
  {
    "method": "GET",
    "url": "/service/values/state"
  },
  {
    "method": "GET",
    "url": "/service/values/type"
  },
  {
    "method": "GET",
    "url": "/State/Components"
  },
  {
    "method": "POST",
    "url": "/State/Components"
  },
  {
    "method": "DELETE",
    "url": "/State/Components"
  },
  {
    "method": "GET",
    "url": "/State/Components/{xname}"
  },
  {
    "method": "PUT",
    "url": "/State/

In [1]:
import os
import yaml
import regex
import json
import re





In [29]:
OPEN_API_FILE = "example_data/swagger_v2.yaml"
TAVERN_FILE_DIR = "example_data/functional"
TAVERN_FILES = []
TAVERN_REGEX = "test_.*.tavern.yaml"
API_TARGET_URLS = ["{hsm_base_url}"]  #use this to specify variables you care about!
TAVERN_CONFIG = "example_data/tavern_global_config.yaml"


# yaml.add_multi_constructor(u'!bool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anything', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)
# yaml.add_multi_constructor(u'!anybool', lambda loader, suffix, node: None, Loader=yaml.SafeLoader)

#if there are unknown tags, ignore them, return None; I am doing this is place of using safeloader which only uses known tags
class SafeLoaderIgnoreUnknown(yaml.SafeLoader):
    def ignore_unknown(self, node):
        return None


SafeLoaderIgnoreUnknown.add_constructor(None, SafeLoaderIgnoreUnknown.ignore_unknown)

# parse all the tavern test files

test_cases = []

for file in os.listdir(TAVERN_FILE_DIR):
    file_path = os.path.join(TAVERN_FILE_DIR, file)
    if os.path.isfile(file_path):
        if regex.search(TAVERN_REGEX, file) is not None:


            with open(file_path) as stream:
                #Im not using safe load, because I have a custom constructor to turn all unknown tags into Nones
                tavern_docs = yaml.load_all(stream, Loader=SafeLoaderIgnoreUnknown)
                for doc in tavern_docs:
                    test = doc["test_name"]

                    for stage in doc["stages"]:
                        test_data = {}
                        test_data["file"] = file
                        test_data["test_name"] = test
                        test_stage = test + " - " + stage["name"]
                        test_data["stage"] = stage["name"]
                        test_data["request_url"] = stage["request"]["url"]
                        test_data["request_method"] = stage["request"]["method"].upper()
                        test_cases.append(test_data)
                        #test_case = (test_stage, stage["request"]["url"], stage["request"]["method"])
                        #print(test_case)
                    # for k,v in doc.items():
                    #     print(k,v)

In [30]:
print(json.dumps(test_cases,indent=2))


[
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can conduct a query for all Nodes in the Component collection",
    "stage": "Verify the expected response fields for all Nodes",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components?type=Node",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Verify the Components collection",
    "stage": "Make sure that a list of Components is returned",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Verify the Components collection",
    "stage": "Make sure that the Component is returned",
    "request_url": "{hsm_base_url}/hsm/v2/State/Components?id={ID}",
    "request_method": "GET"
  },
  {
    "file": "test_components.tavern.yaml",
    "test_name": "Ensure that we can conduct a query for all Node BMCs in the Component collection",
    "stage": "Verif

In [31]:
#filter out any urls that dont have the expected URL string in them.
filtered_test_cases = []
for test_case in test_cases:
    for urls in API_TARGET_URLS:
        if urls in test_case["request_url"]:
            filtered_test_cases.append(test_case)


In [32]:
with open(TAVERN_CONFIG) as stream:
    try:
        config = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        logging.error(exc)
        exit(1)

#todo not sure I need this file?

In [33]:
# request_url = "{hsm_base_url}/hsm/v2/State/Components/{xname}"
# vs  url "/State/Components/{xname}" -> There is no guarantee that the variable is the same.  I think I want to convert all {.*} into a magic string like: {VARIABLE}
#But need to do this AFTER the base url filtering...

for test_case in filtered_test_cases:
    test_case["request_url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", test_case["request_url"])


for endpoint in api_methods:
    endpoint["url"] = regex.sub(r'\{(.*?)\}', "{VARIABLE}", endpoint["url"])



In [34]:
#TODO start to build an endpoints map

endpoints = {}

for ep in api_methods:
    if ep["url"] in endpoints:
        if ep["method"] not in endpoints[ep["url"]]:
            endpoints[ep["url"]].setdefault(ep["method"], [])

    else:
        method = {}
        method[ep["method"]] = []
        endpoints.setdefault(ep["url"], method)
        # endpoint = {}
        # endpoint[ep["method"]] = {}
        # endpoints[ep["url"]] = endpoint

print(json.dumps(endpoints,indent=2))

{
  "/service/ready": {
    "GET": []
  },
  "/service/liveness": {
    "GET": []
  },
  "/service/values": {
    "GET": []
  },
  "/service/values/arch": {
    "GET": []
  },
  "/service/values/class": {
    "GET": []
  },
  "/service/values/flag": {
    "GET": []
  },
  "/service/values/nettype": {
    "GET": []
  },
  "/service/values/role": {
    "GET": []
  },
  "/service/values/subrole": {
    "GET": []
  },
  "/service/values/state": {
    "GET": []
  },
  "/service/values/type": {
    "GET": []
  },
  "/State/Components": {
    "GET": [],
    "POST": [],
    "DELETE": []
  },
  "/State/Components/{VARIABLE}": {
    "GET": [],
    "PUT": [],
    "DELETE": []
  },
  "/State/Components/ByNID/{VARIABLE}": {
    "GET": []
  },
  "/State/Components/BulkStateData": {
    "PATCH": []
  },
  "/State/Components/{VARIABLE}/StateData": {
    "PATCH": []
  },
  "/State/Components/BulkFlagOnly": {
    "PATCH": []
  },
  "/State/Components/{VARIABLE}/FlagOnly": {
    "PATCH": []
  },
  "/Stat

In [35]:


from urllib.parse import urlparse
1 -1
print(urlparse('//www.cwi.nl:80/%7Eguido/Python.html'))
print(urlparse('//www.cwi.nl:80/%7Eguido/Python.html?foo=true'))

#ParseResult(scheme='', netloc='www.cwi.nl:80', path='/%7Eguido/Python.html',            params='', query='', fragment='')
print(urlparse('www.cwi.nl/%7Eguido/Python.html'))
#ParseResult(scheme='', netloc='', path='www.cwi.nl/%7Eguido/Python.html', params='', query='', fragment='')
print(urlparse('help/Python.html'))
print(urlparse('{hsm_base_url}/hsm/v2/State/Components?type=NodeBMC'))

#ParseResult(scheme='', netloc='', path='help/Python.html', params='', query='', fragment='')

ParseResult(scheme='', netloc='www.cwi.nl:80', path='/%7Eguido/Python.html', params='', query='', fragment='')
ParseResult(scheme='', netloc='www.cwi.nl:80', path='/%7Eguido/Python.html', params='', query='foo=true', fragment='')
ParseResult(scheme='', netloc='', path='www.cwi.nl/%7Eguido/Python.html', params='', query='', fragment='')
ParseResult(scheme='', netloc='', path='help/Python.html', params='', query='', fragment='')
ParseResult(scheme='', netloc='', path='{hsm_base_url}/hsm/v2/State/Components', params='', query='type=NodeBMC', fragment='')


In [36]:
#Im seeing a LOT of duplicate data, I must have set some iterator wrongly :(
new_endpoints = dict()
for test_case in filtered_test_cases:
    parsed_url = urlparse(test_case["request_url"])
    method = test_case["request_method"]
    for endpoint, verb in endpoints.items():
        sub_endpoint = re.sub("{VARIABLE}", "[{}a-zA-Z0-9_]*",endpoint)
        sub_endpoint = sub_endpoint + "$"
        if re.search(sub_endpoint, parsed_url.path) is not None and method in verb:
        # if sub in parsed_url.path:
            #verb[method].append(test_case)
            #endpoints[endpoint][verb] = verb
            #need to store it back, this VVV aint quite right
            # print(type(endpoints[endpoint][verb[method]]))
            nd = endpoints[endpoint]
            nd[method].append(test_case)
            new_endpoints[endpoint] = nd
            #print(type(nd[method]))
            #print(type(endpoints[endpoint]))
            #1 -1
            #print("SWAGGER: " , sub_endpoint, "aka", endpoint, " is in TEST URL: ", parsed_url.path, " METHOD ", method, verb )#, test_case)
            #print()
    #if test_case['']

In [40]:
final_endpoints = {}
for endpoint, verb in new_endpoints.items():
    data = {}
    data["methods"] = verb
    data["counts"] = {}

    for method, test_case_list in verb.items():
        data["counts"][method] = len(test_case_list)
    final_endpoints[endpoint] = data


In [41]:
print(json.dumps(final_endpoints,indent=2))


{
  "/State/Components": {
    "methods": {
      "GET": [
        {
          "file": "test_components.tavern.yaml",
          "test_name": "Ensure that we can conduct a query for all Nodes in the Component collection",
          "stage": "Verify the expected response fields for all Nodes",
          "request_url": "{VARIABLE}/hsm/v2/State/Components?type=Node",
          "request_method": "GET"
        },
        {
          "file": "test_components.tavern.yaml",
          "test_name": "Verify the Components collection",
          "stage": "Make sure that a list of Components is returned",
          "request_url": "{VARIABLE}/hsm/v2/State/Components",
          "request_method": "GET"
        },
        {
          "file": "test_components.tavern.yaml",
          "test_name": "Verify the Components collection",
          "stage": "Make sure that the Component is returned",
          "request_url": "{VARIABLE}/hsm/v2/State/Components?id={VARIABLE}",
          "request_method": "GET"
 

In [37]:
print(json.dumps(new_endpoints,indent=2))


{
  "/State/Components": {
    "GET": [
      {
        "file": "test_components.tavern.yaml",
        "test_name": "Ensure that we can conduct a query for all Nodes in the Component collection",
        "stage": "Verify the expected response fields for all Nodes",
        "request_url": "{VARIABLE}/hsm/v2/State/Components?type=Node",
        "request_method": "GET"
      },
      {
        "file": "test_components.tavern.yaml",
        "test_name": "Verify the Components collection",
        "stage": "Make sure that a list of Components is returned",
        "request_url": "{VARIABLE}/hsm/v2/State/Components",
        "request_method": "GET"
      },
      {
        "file": "test_components.tavern.yaml",
        "test_name": "Verify the Components collection",
        "stage": "Make sure that the Component is returned",
        "request_url": "{VARIABLE}/hsm/v2/State/Components?id={VARIABLE}",
        "request_method": "GET"
      },
      {
        "file": "test_components.tavern.yam